# Topic Detection with LDA


[0] __Download Stopwords__


In [5]:
import nltk; nltk.download('stopwords');

[nltk_data] Downloading package stopwords to /home/oso/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[2] __Imports__

In [13]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
"""library for topic modelling, document indexing and similarity
retrieval with large corpora.
"""
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
"""a library for advanced Natural Language Processing in Python
and Cython
"""

# Plotting tools
import pyLDAvis
"""library for interactive topic model visualization.
"""
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

# Logging for gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


[3]  __Importing Data and Peeking at what we have__

In [22]:
# load Dataset
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')

print(df.target_names.unique())
print("\nData Overview is :{}".format(df.head()))
print("\nData Shape is :{}".format(df.shape))

['rec.autos' 'comp.sys.mac.hardware' 'rec.motorcycles' 'misc.forsale'
 'comp.os.ms-windows.misc' 'alt.atheism' 'comp.graphics'
 'rec.sport.baseball' 'rec.sport.hockey' 'sci.electronics' 'sci.space'
 'talk.politics.misc' 'sci.med' 'talk.politics.mideast'
 'soc.religion.christian' 'comp.windows.x' 'comp.sys.ibm.pc.hardware'
 'talk.politics.guns' 'talk.religion.misc' 'sci.crypt']

Data Overview is :                                                content  target  \
0     From: lerxst@wam.umd.edu (where's my thing)\nS...       7   
1     From: guykuo@carson.u.washington.edu (Guy Kuo)...       4   
10    From: irwin@cmptrc.lonestar.org (Irwin Arnstei...       8   
100   From: tchen@magnus.acs.ohio-state.edu (Tsung-K...       6   
1000  From: dabl2@nlm.nih.gov (Don A.B. Lindbergh)\n...       2   

                 target_names  
0                   rec.autos  
1       comp.sys.mac.hardware  
10            rec.motorcycles  
100              misc.forsale  
1000  comp.os.ms-windows.misc  

Data 

In [28]:
# convert to list
data = df.content.values.tolist()

# remove emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# remove newline characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# remove distracting slingle quotes
data = [re.sub("\''", "", sent) for sent in data]

pprint(data[:1])


["From: (where's my thing) Subject: WHAT car is this!? Nntp-Posting-Host: "
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']
